In [2]:
import gtsam

In [3]:
# Parameters
scenes = ['indoor','outdoor','transportation']
scene_sym = gtsam.symbol('s',0)
observations = ['indoor','outdoor','transportation']
obs_sym = gtsam.symbol('o',0)

## Prior

In [107]:
# Define prior
categorical_prior = gtsam.DiscreteDistribution([scene_sym,len(scenes)],[1., 1., 1.])
post = categorical_prior
print(categorical_prior)

Discrete Prior
 P( s0 ):
 Leaf 0.33333333




In [114]:
post.pmf()

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

## Sensor model (Conditional Probability Table)

In [5]:
sensor_model = gtsam.DiscreteConditional([obs_sym,len(observations)],[[scene_sym,len(scenes)]],"5/4/1 1/8/1 2/2/4")
print(sensor_model)

Discrete Conditional
 P( o0 | s0 ):
 Choice(s0) 
 0 Choice(o0) 
 0 0 Leaf  0.5
 0 1 Leaf  0.4
 0 2 Leaf  0.1
 1 Choice(o0) 
 1 0 Leaf  0.1
 1 1 Leaf  0.8
 1 2 Leaf  0.1
 2 Choice(o0) 
 2 0 Leaf 0.25
 2 1 Leaf 0.25
 2 2 Leaf  0.5




## Measurement

In [94]:
obs = gtsam.DiscreteDistribution([obs_sym,len(observations)],"0/2/0")

## Update - argmax likelihood

In [95]:
likelihood = sensor_model.likelihood(obs.argmax())
print(likelihood)
likelihood.enumerate()

DecisionTreeFactor
 f[ (s0,3), ]
 Choice(s0) 
 0 Leaf  0.4
 1 Leaf  0.8
 2 Leaf 0.25



[(DiscreteValues{8286623314361712640: 0}, 0.4),
 (DiscreteValues{8286623314361712640: 1}, 0.8),
 (DiscreteValues{8286623314361712640: 2}, 0.25)]

In [96]:
post = gtsam.DiscreteDistribution(likelihood*post)
print(post)

Discrete Prior
 P( s0 ):
 Choice(s0) 
 0 Leaf 0.99999046
 1 Leaf 1.3107075e-11
 2 Leaf 9.5366522e-06




## THE ACTUAL CORRECT WAY TO UPDATE TODO

In [101]:
obs_tree_factor = gtsam.DecisionTreeFactor(obs)
sensor_model_factor = gtsam.DecisionTreeFactor(sensor_model)
likelihood = (obs_tree_factor*sensor_model_factor).sum(1)

In [106]:
post = gtsam.DiscreteDistribution(likelihood*post)
# post = gtsam.DiscreteDistribution(likelihood*post)
print(post)

Discrete Prior
 P( s0 ):
 Choice(s0) 
 0 Leaf 0.99999909
 1 Leaf 4.1943002e-10
 2 Leaf 9.0949387e-07




In [20]:
help (gtsam.DiscreteValues)

Help on class DiscreteValues in module gtsam.gtsam:

class DiscreteValues(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      DiscreteValues
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __bool__(...)
 |      __bool__(self: gtsam.gtsam.DiscreteValues) -> bool
 |      
 |      Check whether the map is nonempty
 |  
 |  __contains__(...)
 |      __contains__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __contains__(self: gtsam.gtsam.DiscreteValues, arg0: int) -> bool
 |      
 |      2. __contains__(self: gtsam.gtsam.DiscreteValues, arg0: object) -> bool
 |  
 |  __delitem__(...)
 |      __delitem__(self: gtsam.gtsam.DiscreteValues, arg0: int) -> None
 |  
 |  __getitem__(...)
 |      __getitem__(self: gtsam.gtsam.DiscreteValues, arg0: int) -> int
 |  
 |  __init__(...)
 |      __init__(self: gtsam.gtsam.DiscreteValues) -> None
 |  
 |  __iter__(...)
 |      __iter__(self: gtsam.gtsam.Dis

In [19]:
help (gtsam.DiscreteConditional)

Help on class DiscreteConditional in module gtsam.gtsam:

class DiscreteConditional(DecisionTreeFactor)
 |  Method resolution order:
 |      DiscreteConditional
 |      DecisionTreeFactor
 |      DiscreteFactor
 |      Factor
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: gtsam.gtsam.DiscreteConditional) -> None
 |      
 |      2. __init__(self: gtsam.gtsam.DiscreteConditional, nFrontals: int, f: gtsam.gtsam.DecisionTreeFactor) -> None
 |      
 |      3. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], spec: str) -> None
 |      
 |      4. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], parents: gtsam.gtsam.DiscreteKeys, spec: str) -> None
 |      
 |      5. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], parents: List[Tuple[int, int]], spec: str) ->

In [18]:
help (gtsam.DecisionTreeFactor)

Help on class DecisionTreeFactor in module gtsam.gtsam:

class DecisionTreeFactor(DiscreteFactor)
 |  Method resolution order:
 |      DecisionTreeFactor
 |      DiscreteFactor
 |      Factor
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(...)
 |      __call__(self: gtsam.gtsam.DecisionTreeFactor, arg0: gtsam::DiscreteValues) -> float
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: gtsam.gtsam.DecisionTreeFactor) -> None
 |      
 |      2. __init__(self: gtsam.gtsam.DecisionTreeFactor, key: Tuple[int, int], spec: List[float]) -> None
 |      
 |      3. __init__(self: gtsam.gtsam.DecisionTreeFactor, key: Tuple[int, int], table: str) -> None
 |      
 |      4. __init__(self: gtsam.gtsam.DecisionTreeFactor, keys: gtsam.gtsam.DiscreteKeys, table: List[float]) -> None
 |      
 |      5. __init__(self: gtsam.gtsam.DecisionTreeFactor, keys: gtsam.gts